# 서포트 벡터 머신

## 선형 SVM 분류

경계에 위치한 샘플에 의해 전적으로 결정된다. 이러한 샘플들 -> 서포트 벡터

모든 샘플이 도로 바깥쪽에 올바르게 부뉼 -> 하드 마진 분류

하드마진은 이상치에 민감

하이퍼파라미터 C로 마진 넓이 조정
- 낮게 설정할 수록 마진 높아짐
- 과대 적합시 C를 감소시켜 규제

In [2]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:,(2,3)]
y = (iris["target"]==2).astype(np.float64)

svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge")),
])
svm_clf.fit(X,y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge'))])

In [4]:
svm_clf.predict([[5.5,1.7]])

array([1.])

다른 방법으로 SGDClassifier모델 사용하는 법도 있다. (확률적 경사하강법 사용)

LinearSVC는 규제에 편향을 포함시키기 때문에 평균을 빼서 중앙에 맞춰야 한다.
loss 매개변수도 hinge로 지정한다.

## 비선형 SVM 분류

특성을 추가하여 선형적으로 구분되는 데이터셋 만들수 있다.

In [6]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples=100, noise=0.15)
polynomial_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler",StandardScaler()),
    ("svm_clf",LinearSVC(C=10,loss="hinge"))
])
polynomial_svm_clf.fit(X,y)

Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('svm_clf', LinearSVC(C=10, loss='hinge'))])

### 다항식 커널

* 커널 트릭

In [8]:
from sklearn.svm import SVC
poly_kernel_svm_clf = Pipeline([
    ("scaler",StandardScaler()),
    ("svm_clf",SVC(kernel="poly", degree=3, coef0=1, C=5))
])
poly_kernel_svm_clf.fit(X,y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

### 유사도 특성

가우시안 RBF(방사 기저 함수)
감마는 0보다 커야하며 값이 작을수록 폭이 넓은 종 모양,
l은 랜드마크 지점

* 가우시안 RBF 커널을 사용한 SVC 모델

In [9]:
rbf_kernel_svm_clf = Pipeline([
    ("scaler",StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
])
rbf_kernel_svm_clf.fit(X,y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=0.001, gamma=5))])

#### LinearSVC
- $$O(m X n)$$
- 커널트릭 지원 X, 그렇지만 훈련 샘플과 특성수에 거의 선형적으로 증가
- 정밀도는 허용오차 tol로 조정, 수행시간과 반비례

#### SVC
- $$O(m^2 X n) ~ O(m^3 X n)$$
- 커널 트릭 알고리즘을 구현한 libsvm 라이브러리 기반
- 희소특성(특성이 몇 개 없는 경우)인 경우에 잘 확장된다.

## SVM 회귀

- 분류와 달리 마진 안에 가능한 많은 샘플이 들어오도록 학습
- 폭은 epsilon으로 조절
- 마진 안에 훈련샘플이 추가되어도 예측에 영향이 없어서 epsilon에 민감하지 않다고 한다.

In [10]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X,y)

LinearSVR(epsilon=1.5)

- C가 크면 규제가 적어지고, C가 작아지면 규제가 많아진다.

In [11]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100,epsilon=0.1)
svm_poly_reg.fit(X,y)

SVR(C=100, degree=2, kernel='poly')